In [0]:
import numpy as np
import torch
from torch import nn
import torch.nn.functional as F
import sys
import numpy as np
import pandas as pd


In [0]:
# http://pytorch.org/
from os.path import exists
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.4.1-{platform}-linux_x86_64.whl torchvision
import torch

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')
!ls "/content/gdrive/My Drive/PytorchScholarshipChallenge"


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
' ChallengeLab.ipynb'   flower_data.zip        requirementlist.txt
 classifier_flower.pt   flowerResnet.ipynb     TextCreation.ipynb
 classifier.pth         names.txt	       Untitled0.ipynb
 flower_classifier.pt   requirementlist.gdoc


In [0]:

# open text file and read in data as `text`
with open('/content/gdrive/My Drive/PytorchScholarshipChallenge/names.txt', 'r') as f:
    text = f.read()

In [5]:
print(text[:100])

﻿Aaron, Hank
Abagnale, Frank
Abbey, Edward
Abel, Reuben
Abelson, Hal
Abourezk, James
Abrams, Creight


In [6]:
chars = tuple(set(text))
int2char = dict(enumerate(chars))
char2int = {ch:ii for ii, ch in int2char.items()}

encoded = np.array([char2int[ch] for ch in text])
print(encoded)

[62 32 77 ... 64 22 34]


In [7]:
encoded[:100]

array([62, 32, 77, 23, 10, 86, 40, 51, 18, 77, 86, 22, 34, 32,  4, 77, 47,
       86, 77, 87, 64, 40, 51,  6, 23, 77, 86, 22, 34, 32,  4,  4, 64, 72,
       40, 51, 80, 76, 81, 77, 23, 76, 34, 32,  4, 64, 87, 40, 51,  3, 64,
       12,  4, 64, 86, 34, 32,  4, 64, 87, 61, 10, 86, 40, 51, 18, 77, 87,
       34, 32,  4, 10, 12, 23, 64, 21, 22, 40, 51, 19, 77, 27, 64, 61, 34,
       32,  4, 23, 77, 27, 61, 40, 51, 53, 23, 64, 31, 47, 89, 36])

In [0]:
def one_hot_encode(arr, n_labels):
    
    # Initialize the the encoded array
    one_hot = np.zeros((np.multiply(*arr.shape), n_labels), dtype=np.float32)
    
    # Fill the appropriate elements with ones
    one_hot[np.arange(one_hot.shape[0]), arr.flatten()] = 1.
    
    # Finally reshape it to get back to the original array
    one_hot = one_hot.reshape((*arr.shape, n_labels))
    
    return one_hot

In [9]:
test_seq = np.array([[3, 5, 1]])
one_hot = one_hot_encode(test_seq, 8)

print(one_hot)

[[[0. 0. 0. 1. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 1. 0. 0.]
  [0. 1. 0. 0. 0. 0. 0. 0.]]]


In [0]:
def get_batches(arr, batch_size, seq_length):
    '''Create a generator that returns batches of size
       batch_size x seq_length from arr.
       
       Arguments
       ---------
       arr: Array you want to make batches from
       batch_size: Batch size, the number of sequences per batch
       seq_length: Number of encoded chars in a sequence
    '''
    
    batch_size_total = batch_size * seq_length
    # total number of batches we can make
    n_batches = len(arr)//batch_size_total
    
    # Keep only enough characters to make full batches
    arr = arr[:n_batches * batch_size_total]
    # Reshape into batch_size rows
    arr = arr.reshape((batch_size, -1))
    
    # iterate through the array, one sequence at a time
    for n in range(0, arr.shape[1], seq_length):
        # The features
        x = arr[:, n:n+seq_length]
        # The targets, shifted by one
        y = np.zeros_like(x)
        try:
            y[:, :-1], y[:, -1] = x[:, 1:], arr[:, n+seq_length]
        except IndexError:
            y[:, :-1], y[:, -1] = x[:, 1:], arr[:, 0]
        yield x, y

In [0]:
batches = get_batches(encoded, 8, 50)
x, y = next(batches)

In [12]:
# printing out the first 10 items in a sequence
print('x\n', x[:10, :10])
print('\ny\n', y[:10, :10])

x
 [[62 32 77 23 10 86 40 51 18 77]
 [ 2 23 77 86 76 10 40 51  1 77]
 [34 53 12 27 27 31 86 47 61 40]
 [77 86 68 73 10 12 31 61 34 56]
 [64 34 19 77 88 22 61 10 86 40]
 [51 52 31 88 88 10 87 10 34  1]
 [36 10 86 40 51 56 64 10 23 47]
 [63 12 61 77 86 34 63 10 10 86]]

y
 [[32 77 23 10 86 40 51 18 77 86]
 [23 77 86 76 10 40 51  1 77 23]
 [53 12 27 27 31 86 47 61 40 51]
 [86 68 73 10 12 31 61 34 56 77]
 [34 19 77 88 22 61 10 86 40 51]
 [52 31 88 88 10 87 10 34  1 77]
 [10 86 40 51 56 64 10 23 47 64]
 [12 61 77 86 34 63 10 10 86 47]]


In [13]:
# check if GPU is available
train_on_gpu = torch.cuda.is_available()
if(train_on_gpu):
    print('Training on GPU!')
else: 
    print('No GPU available, training on CPU; consider making n_epochs very small.')

No GPU available, training on CPU; consider making n_epochs very small.


In [0]:
class CharRNN(nn.Module):
    
    def __init__(self, tokens, n_hidden=256, n_layers=2,
                               drop_prob=0.5, lr=0.001):
        super().__init__()
        self.drop_prob = drop_prob
        self.n_layers = n_layers
        self.n_hidden = n_hidden
        self.lr = lr
        
        # creating character dictionaries
        self.chars = tokens
        self.int2char = dict(enumerate(self.chars))
        self.char2int = {ch: ii for ii, ch in self.int2char.items()}
        
        ## TODO: define the LSTM
        self.lstm = nn.LSTM(len(self.chars), n_hidden, n_layers, 
                            dropout=drop_prob, batch_first=True)
        
        ## TODO: define a dropout layer
        self.dropout = nn.Dropout(drop_prob)
        
        ## TODO: define the final, fully-connected output layer
        self.fc = nn.Linear(n_hidden, len(self.chars))
      
    
    def forward(self, x, hidden):
        ''' Forward pass through the network. 
            These inputs are x, and the hidden/cell state `hidden`. '''
                
        ## TODO: Get the outputs and the new hidden state from the lstm
        r_output, hidden = self.lstm(x, hidden)
        
        ## TODO: pass through a dropout layer
        out = self.dropout(r_output)
        
        # Stack up LSTM outputs using view
        # you may need to use contiguous to reshape the output
        out = out.contiguous().view(-1, self.n_hidden)
        
        ## TODO: put x through the fully-connected layer
        out = self.fc(out)
        
        # return the final output and the hidden state
        return out, hidden
    
    
    def init_hidden(self, batch_size):
        ''' Initializes hidden state '''
        # Create two new tensors with sizes n_layers x batch_size x n_hidden,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data
        
        if (train_on_gpu):
            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda(),
                  weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda())
        else:
            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_(),
                      weight.new(self.n_layers, batch_size, self.n_hidden).zero_())
        
        return hidden

In [0]:
def train(net, data, epochs=10, batch_size=10, seq_length=50, lr=0.001, clip=5, val_frac=0.1, print_every=10):
    ''' Training a network 
    
        Arguments
        ---------
        
        net: CharRNN network
        data: text data to train the network
        epochs: Number of epochs to train
        batch_size: Number of mini-sequences per mini-batch, aka batch size
        seq_length: Number of character steps per mini-batch
        lr: learning rate
        clip: gradient clipping
        val_frac: Fraction of data to hold out for validation
        print_every: Number of steps for printing training and validation loss
    
    '''
    net.train()
    
    opt = torch.optim.Adam(net.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()
    
    # create training and validation data
    val_idx = int(len(data)*(1-val_frac))
    data, val_data = data[:val_idx], data[val_idx:]
    
    if(train_on_gpu):
        net.cuda()
    
    counter = 0
    n_chars = len(net.chars)
    for e in range(epochs):
        # initialize hidden state
        h = net.init_hidden(batch_size)
        
        for x, y in get_batches(data, batch_size, seq_length):
            counter += 1
            
            # One-hot encode our data and make them Torch tensors
            x = one_hot_encode(x, n_chars)
            inputs, targets = torch.from_numpy(x), torch.from_numpy(y)
            
            if(train_on_gpu):
                inputs, targets = inputs.cuda(), targets.cuda()

            # Creating new variables for the hidden state, otherwise
            # we'd backprop through the entire training history
            h = tuple([each.data for each in h])

            # zero accumulated gradients
            net.zero_grad()
            
            # get the output from the model
            output, h = net(inputs, h)
            
            # calculate the loss and perform backprop
            loss = criterion(output, targets.view(batch_size*seq_length))
            loss.backward()
            # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
            nn.utils.clip_grad_norm_(net.parameters(), clip)
            opt.step()
            
            # loss stats
            if counter % print_every == 0:
                # Get validation loss
                val_h = net.init_hidden(batch_size)
                val_losses = []
                net.eval()
                for x, y in get_batches(val_data, batch_size, seq_length):
                    # One-hot encode our data and make them Torch tensors
                    x = one_hot_encode(x, n_chars)
                    x, y = torch.from_numpy(x), torch.from_numpy(y)
                    
                    # Creating new variables for the hidden state, otherwise
                    # we'd backprop through the entire training history
                    val_h = tuple([each.data for each in val_h])
                    
                    inputs, targets = x, y
                    if(train_on_gpu):
                        inputs, targets = inputs.cuda(), targets.cuda()

                    output, val_h = net(inputs, val_h)
                    val_loss = criterion(output, targets.view(batch_size*seq_length))
                
                    val_losses.append(val_loss.item())
                
                net.train() # reset to train mode after iterationg through validation data
                
                print("Epoch: {}/{}...".format(e+1, epochs),
                      "Step: {}...".format(counter),
                      "Loss: {:.4f}...".format(loss.item()),
                      "Val Loss: {:.4f}".format(np.mean(val_losses)))

In [16]:
# define and print the net
n_hidden=512
n_layers=2

net = CharRNN(chars, n_hidden, n_layers)
print(net)

CharRNN(
  (lstm): LSTM(90, 512, num_layers=2, batch_first=True, dropout=0.5)
  (dropout): Dropout(p=0.5)
  (fc): Linear(in_features=512, out_features=90, bias=True)
)


In [17]:
batch_size = 128
seq_length = 5
n_epochs = 20 # start smaller if you are just testing initial behavior

# train the model
train(net, encoded, epochs=n_epochs, batch_size=batch_size, seq_length=seq_length, lr=0.001, print_every=10)

Epoch: 1/20... Step: 10... Loss: 3.7105... Val Loss: 3.7070
Epoch: 1/20... Step: 20... Loss: 3.5198... Val Loss: 3.6280
Epoch: 1/20... Step: 30... Loss: 3.4624... Val Loss: 3.5892
Epoch: 1/20... Step: 40... Loss: 3.4613... Val Loss: 3.6021
Epoch: 2/20... Step: 50... Loss: 3.4638... Val Loss: 3.5867
Epoch: 2/20... Step: 60... Loss: 3.3585... Val Loss: 3.5638
Epoch: 2/20... Step: 70... Loss: 3.4840... Val Loss: 3.5566
Epoch: 2/20... Step: 80... Loss: 3.4437... Val Loss: 3.5527
Epoch: 2/20... Step: 90... Loss: 3.4412... Val Loss: 3.5309
Epoch: 3/20... Step: 100... Loss: 3.3465... Val Loss: 3.4643
Epoch: 3/20... Step: 110... Loss: 3.3404... Val Loss: 3.3606
Epoch: 3/20... Step: 120... Loss: 3.1257... Val Loss: 3.2734
Epoch: 3/20... Step: 130... Loss: 2.9134... Val Loss: 3.1617
Epoch: 4/20... Step: 140... Loss: 2.9095... Val Loss: 3.0992
Epoch: 4/20... Step: 150... Loss: 2.8075... Val Loss: 3.0164
Epoch: 4/20... Step: 160... Loss: 2.7733... Val Loss: 2.9479
Epoch: 4/20... Step: 170... Loss:

In [0]:

# change the name, for saving multiple files
model_name = 'rnn_20_epoch.net'

checkpoint = {'n_hidden': net.n_hidden,
              'n_layers': net.n_layers,
              'state_dict': net.state_dict(),
              'tokens': net.chars}

with open(model_name, 'wb') as f:
    torch.save(checkpoint, f)

In [0]:
def predict(net, char, h=None, top_k=None):
        ''' Given a character, predict the next character.
            Returns the predicted character and the hidden state.
        '''
        
        # tensor inputs
        x = np.array([[net.char2int[char]]])
        x = one_hot_encode(x, len(net.chars))
        inputs = torch.from_numpy(x)
        
        if(train_on_gpu):
            inputs = inputs.cuda()
        
        # detach hidden state from history
        h = tuple([each.data for each in h])
        # get the output of the model
        out, h = net(inputs, h)

        # get the character probabilities
        p = F.softmax(out, dim=1).data
        if(train_on_gpu):
            p = p.cpu() # move to cpu
        
        # get top characters
        if top_k is None:
            top_ch = np.arange(len(net.chars))
        else:
            p, top_ch = p.topk(top_k)
            top_ch = top_ch.numpy().squeeze()
        
        # select the likely next character with some element of randomness
        p = p.numpy().squeeze()
        char = np.random.choice(top_ch, p=p/p.sum())
        
        # return the encoded value of the predicted char and the hidden state
        return net.int2char[char], h

In [0]:
def sample(net, size, prime='The', top_k=None):
        
    if(train_on_gpu):
        net.cuda()
    else:
        net.cpu()
    
    net.eval() # eval mode
    
    # First off, run through the prime characters
    chars = [ch for ch in prime]
    h = net.init_hidden(1)
    for ch in prime:
        char, h = predict(net, ch, h, top_k=top_k)

    chars.append(char)
    
    # Now pass in the previous character and get a new one
    for ii in range(size):
        char, h = predict(net, chars[-1], h, top_k=top_k)
        chars.append(char)

    return ''.join(chars)

In [51]:
nameslist = sample(net, 1000, prime='anu', top_k=20)
# print(nameslist)
newnames = nameslist.split('\n')
newn = sorted(newnames)
print
print(newn)


['', 'Aldus, Anthumis', 'Allen, Duigel', 'An Hote, Oldernim', 'Andes, Juna', 'Bamte, Edi', 'Beblry, Miico', 'Berchald, Rosilor', 'Bicindon, Chrisdas', 'Biske, Ker', 'Biyneer, Alle', 'Blanigzin, Corby', 'Bohpraog, Josela)', 'Bullen, Bert', 'Buochlad, Jack', 'Burlon, Edico', 'Buurko f, He Eng', 'Callera', 'Calnesk, Anch', 'Careadedit', 'Cevelth, Margis', 'Collys, Loff', 'Corbingelli, Roterni', 'Crardisk, Jr.', 'Darycatrick', 'Dorkeg, Jebe', 'Eablitoveerath', 'Frack, Heberte', 'Fuxghonanterike', 'Gel-Ho', 'Genry', 'Giangod, Wilm', 'Grichen, Petrale', 'Gudland, Helury Friece', 'Heem, Alison', 'Hikrissrey, Trompsonerc', 'Hiyneen, Muy', 'Hour, Eric', 'Howince, Lomine', 'Hriigero II, A.', 'Hripnermon, Francer)', 'Jinsonton, Duvady', 'Jomen, Garrda', 'Kank, Weldy', 'Kenebart, Mevin', 'Kunskav, Harie', 'Lratne, Curlenter)', 'Mascicess, John Ocf', 'Matwiel, Steve', 'McFrenk, Salles Rone', 'Merray, Jabe', 'Mibtan, Bary', 'Misoneono, Engere', 'Mudt, John', 'Nass, Richard', 'Noin, Thomas', 'Odwiage